# Forward and backward passes

In [ ]:
#| default_exp dlff

In [ ]:
#| hide
%load_ext autoreload
%autoreload 2

In [ ]:
from dl_from_foundations.dlff import *

In [ ]:
#| export 
from pathlib import Path
import gzip
import pickle
import torch
from torch import tensor

def get_data():
    path = Path('../mnist.pkl.gz')
    with gzip.open(path, 'rb') as f:
        ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding='latin-1')
    return map(tensor, (x_train,y_train,x_valid,y_valid))

def normalise(x, m, s):
    return (x-m)/s
    

In [ ]:
x_train,y_train,x_valid,y_valid = get_data()

In [ ]:
x_train.shape

torch.Size([50000, 784])

In [ ]:
train_mean, train_std = x_train.mean(), x_train.std()
train_mean, train_std

(tensor(0.1304), tensor(0.3073))

In [ ]:
x_train = normalise(x_train, train_mean, train_std)
x_valid = normalise(x_valid, train_mean, train_std)

In [ ]:
train_mean, train_std = x_train.mean(), x_train.std()
train_mean, train_std

(tensor(2.1425e-08), tensor(1.))

In [ ]:
#export
def test_near_zero(a,tol=1e-3): assert a.abs()<tol, f"Near zero: {a}"

In [ ]:
test_near_zero(x_train.mean())
test_near_zero(1-x_train.std())

In [ ]:
n,m = x_train.shape
c = y_train.max()+1
n,m,c

(50000, 784, tensor(10))

## Foundation version
### Basic architecture

In [ ]:
# num hidden
nh = 50

In [ ]:
import math
w1 = torch.randn(m,nh)/math.sqrt(m)
b1 = torch.zeros(nh)
w2 = torch.randn(nh,1)/math.sqrt(nh)
b2 = torch.zeros(1)

In [ ]:
def lin(x, w, b): return x@w + b

In [ ]:
x_valid.mean(), x_valid.std()

(tensor(-0.0059), tensor(0.9924))

In [ ]:
# test_near_zero(x_valid.mean())
# test_near_zero(x_valid.std())

AssertionError: Near zero: -0.005850578658282757

In [ ]:
t = lin(x_valid, w1, b1)

In [ ]:
t.mean(), t.std()

(tensor(-0.1160), tensor(1.0217))

In [ ]:
# test_near_zero(t.mean())
# test_near_zero(1-t.std())

AssertionError: Near zero: -0.11602797359228134

In [ ]:
def relu(x): return x.clamp_min(0.)

In [ ]:
t = relu(lin(x_valid,w1,b1))

In [ ]:
t.mean(), t.std()

(tensor(0.3445), tensor(0.5667))

In [ ]:
#kaiming init / he init for relu
w1 = torch.randn(m,nh)*math.sqrt(2/m)

In [ ]:
w1.mean(), w1.std()

(tensor(0.0001), tensor(0.0505))

In [ ]:
t = relu(lin(x_valid, w1, b1))

In [ ]:
t.mean(), t.std()

(tensor(0.5939), tensor(0.8526))

In [ ]:
from torch.nn import init

In [ ]:
w1 = torch.zeros(m,nh)
init.kaiming_normal(w1, mode='fan_out')
t = relu(lin(x_valid, w1, b1))

/tmp/ipykernel_11311/2157128107.py:2: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  init.kaiming_normal(w1, mode='fan_out')


In [ ]:
t.mean(), t.std()

(tensor(0.6070), tensor(0.8412))

In [ ]:
w1.mean(), w1.std()

(tensor(-0.0002), tensor(0.0506))

In [ ]:
def relu(x): return x.clamp_min(0.) - 0.5

In [ ]:
w1 = torch.randn(m,nh) * math.sqrt(2./m)
t1 = relu(lin(x_valid, w1, b1))

In [ ]:
t1.mean(), t1.std()

(tensor(0.0640), tensor(0.8054))

mean of first layer is closer to 0 when we subtract 0.5 from the output of relu

In [ ]:
def model(xb):
    l1 = relu(lin(xb, w1, b1))
    l2 = relu(lin(l1, w2, b2))
    return l2

In [ ]:
%timeit -n 10 _=model(x_valid)

12.7 ms ± 4.93 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [ ]:
assert model(x_valid).shape==torch.Size([x_valid.shape[0], 1])

### Loss function MSE

In [ ]:
#| export
def mse(output, target):
    return (output.squeeze(-1) - target).pow(2).mean()

In [ ]:
y_train, y_valid = y_train.float(), y_valid.float()

In [ ]:
preds = model(x_train)

In [ ]:
preds.shape

torch.Size([50000, 1])

In [ ]:
mse(preds, y_train)

tensor(30.2497)

### Gradients and backward pass

In [ ]:
def mse_grad(inp, targ): 
    # grad of loss with respect to output of previous layer
    inp.g = 2. * (inp.squeeze() - targ).unsqueeze(-1) / inp.shape[0]

In [ ]:
def relu_grad(inp, out):
    # grad of relu with respect to input activations
    inp.g = (inp>0).float() * out.g

In [ ]:
def lin_grad(inp, out, w, b):
    # grad of matmul with respect to input
    inp.g = out.g @ w.t()
    w.g = (inp.unsqueeze(-1) * out.g.unsqueeze(1)).sum(0)
    b.g = out.g.sum(0)

In [ ]:
def forward_and_backward(input, target):
    l1 = input @ w1 + b1
    l2 = relu(l1)
    out = l2 @ w2 + b2
    
    loss = mse(out, target)
    
    #backward pass
    mse_grad(out, target)
    lin_grad(l2, out, w2, b2)
    relu_grad(l1, l2)
    lin_grad(input, l1, w1, b1)
    

In [ ]:
forward_and_backward(x_train, y_train)

In [ ]:
#saving gradients for testing
w1g = w1.g.clone()
b1g = b1.g.clone()
w2g = w2.g.clone()
b2g = b2.g.clone()
ig = x_train.g.clone()

In [ ]:
#tesing the gradients using pytorch
xt2 = x_train.clone().requires_grad_(True)
w12 = w1.clone().requires_grad_(True)
b12 = b1.clone().requires_grad_(True)
w22 = w2.clone().requires_grad_(True)
b22 = b2.clone().requires_grad_(True)

In [ ]:
def forward(inp, target):
    l1 = inp @ w12 + b12
    l2 = relu(l1)
    out = l2 @ w22 + b22
    
    return mse(out, target)

In [ ]:
loss = forward(xt2, y_train)

In [ ]:
loss.backward()

In [ ]:
w12g = w12.grad
b12g = b12.grad
w22g = w22.grad
b22g = b22.grad
xt2g = xt2.grad

In [ ]:
test_near(w12g, w1g)
test_near(b12g, b1g)
test_near(w22g, w2g)
test_near(b22g, b2g)
test_near(xt2g, ig)

## Refactor model

In [ ]:
class Relu:
    def __call__(self, inp):
        self.inp = inp
        self.out = self.inp.clamp_min(0.) - 0.5
        return self.out
    
    def backward(self):
        self.inp.g = (self.inp>0).float() * self.out.g

In [ ]:
class Lin:
    def __init__(self, w, b):
        self.w = w
        self.b = b
    def __call__(self,inp):
        self.inp = inp
        self.out = self.inp @ self.w + self.b
        return self.out
    
    def backward(self):
        self.inp.g = self.out.g @ self.w.t()
        self.w.g = (self.inp.unsqueeze(-1) * self.out.g.unsqueeze(1)).sum(0)
        self.b.g = self.out.g.sum(0)

In [ ]:
class MSE:
    def __call__(self, inp, targ):
        self.inp = inp
        self.targ = targ
        self.out = (self.inp.squeeze(-1) - targ).pow(2).mean()
        return self.out
    
    def backward(self):
        self.inp.g = 2. * (self.inp.squeeze() - self.targ).unsqueeze(-1) / self.inp.shape[0]        

In [ ]:
class Model:
    def __init__(self, w1, b1, w2, b2):
        self.layers = [Lin(w1,b1), Relu(), Lin(w2,b2)]
        self.loss = MSE()
    
    def __call__(self, x, y):
        for l in self.layers: x = l(x)
        return self.loss(x, y)
    
    def backward(self):
        self.loss.backward()
        for l in reversed(self.layers): l.backward()

In [ ]:
w1.g,b1.g,w2.g,b2.g = [None]*4
model = Model(w1, b1, w2, b2)

In [ ]:
%time loss = model(x_train, y_train)

CPU times: user 995 ms, sys: 1.94 ms, total: 997 ms
Wall time: 134 ms


In [ ]:
%time model.backward()

CPU times: user 5.5 s, sys: 5.71 s, total: 11.2 s
Wall time: 1.64 s


In [ ]:
test_near(w2g, w2.g)
test_near(b2g, b2.g)
test_near(w1g, w1.g)
test_near(b1g, b1.g)
test_near(ig, x_train.g)

## Module.forward()

In [ ]:
class Module:
    def __call__(self, *args):
        self.args = args
        self.out = self.forward(*args)
        return self.out
    def forward(self): raise Exception('not implemented')
    def backward(self): self.bwd(self.out, *self.args)

In [ ]:
class Relu(Module):
    def forward(self, inp): return inp.clamp_min(0.) - 0.5
    def bwd(self, out, inp): inp.g = (inp>0).float() * out.g

In [ ]:
class Lin(Module):
    def __init__(self, w, b): self.w,self.b = w,b
        
    def forward(self, inp): return inp@self.w + self.b
    
    def bwd(self, out, inp):
        inp.g = out.g @ self.w.t()
        self.w.g = inp.t() @ out.g
        self.b.g = out.g.sum(0)

In [ ]:
class Mse(Module):
    def forward (self, inp, targ): return (inp.squeeze() - targ).pow(2).mean()
    def bwd(self, out, inp, targ): inp.g = 2*(inp.squeeze()-targ).unsqueeze(-1) / targ.shape[0]


In [ ]:
class Model():
    def __init__(self):
        self.layers = [Lin(w1,b1), Relu(), Lin(w2,b2)]
        self.loss = Mse()
        
    def __call__(self, x, targ):
        for l in self.layers: x = l(x)
        return self.loss(x, targ)
    
    def backward(self):
        self.loss.backward()
        for l in reversed(self.layers): l.backward()

In [ ]:
w1.g,b1.g,w2.g,b2.g = [None]*4
model = Model()

In [ ]:
%time loss = model(x_train, y_train)


CPU times: user 860 ms, sys: 0 ns, total: 860 ms
Wall time: 121 ms


In [ ]:
%time model.backward()


CPU times: user 830 ms, sys: 147 ms, total: 977 ms
Wall time: 139 ms


In [ ]:
test_near(w2g, w2.g)
test_near(b2g, b2.g)
test_near(w1g, w1.g)
test_near(b1g, b1.g)
test_near(ig, x_train.g)

## nn.Linear and nn.Module

In [ ]:
#| export
from torch import nn

In [ ]:
class Model(nn.Module):
    def __init__(self, n_in, n_h, n_out):
        super().__init__()
        self.layers = [nn.Linear(n_in, n_h), nn.ReLU(), nn.Linear(n_h, n_out)]
        self.loss = mse
    
    def __call__(self, x, targ):
        for l in self.layers: x = l(x)
        print(x.shape, targ.shape)
        print(x.squeeze().shape)
        return self.loss(x, targ)

In [ ]:
model = Model(m, nh, 1)

In [ ]:
%time loss = model(x_train, y_train)


torch.Size([50000, 1]) torch.Size([50000])
torch.Size([50000])
CPU times: user 1.04 s, sys: 0 ns, total: 1.04 s
Wall time: 135 ms
